In [ ]:
# Place %autoreload 2 at the top of your notebook to ensure it applies to all subsequent imports.
%load_ext autoreload
%autoreload 2
 


In [ ]:
from __future__ import absolute_import, division, print_function
import torch
from trainer_end_to_end import Trainer
from options import MonodepthOptions
import time
options = MonodepthOptions()
args = [
    '--num_epochs', '50000',
    '--batch_size', '1',
    '--log_frequency', '10',
    '--save_frequency', '100000',
    '--of_samples',
    '--of_samples_num', '300', 
    # '--of_samples_num', '5', 
    # '--batch_size', '10', 
    '--enable_motion_computation', 
    '--use_raft_flow',
    '--pose_model_type', 'endofast3r', 
    '--pose_model_type', 'posetr_net', 
    '--frame_ids', '0', '-4', '4',
    '--frame_ids', '0', '-1', '1',
    '--dataset', 'StereoMIS',
    '--split_appendix', '_offline',
    '--data_path', '/mnt/nct-zfs/TCO-All/SharedDatasets/StereoMIS_DARES_test/',
    '--dataset', 'DynaSCARED',
    '--split_appendix', '_CaToTi001',
    # '--split_appendix', '_CaToTi101',
    # '--split_appendix', '_CaToTi010',
    '--data_path', '/mnt/cluster/datasets/Surg_oclr_stereo/',
    '--dataset', 'StereoMIS',
    '--split_appendix', '_offline',
    '--data_path', '/mnt/nct-zfs/TCO-All/SharedDatasets/StereoMIS_DARES_test/',
    '--motion_mask_thre_px', '1',
    # '--frame_ids', '0', '-1', '1',
    # '--pose_model_type', 'separate_resnet', 
    # '--log_frequency', '2',
    # '--log_dir', '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/debug_reloc3r_backbone/relocxr',
    # '--debug',
    ]
opts = options.parse_notebook(args)# trainer.set_eval()
# # opts
opts.datasets, opts.frame_ids, opts.pose_model_type

In [ ]:
%tb
# self = trainer
# self.train_dataset.frame_ids, self.opt.frame_ids, opts.frame_ids


In [ ]:
# load model
trainer = Trainer(opts)
print('trainer initialized...')

In [ ]:
# load trained model for test
import os
traiend_models_root = '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/unfzD_SCARED_b3accu1_posetr_net_reloc3r_9D_OPTTranse01_ROTAng01_Loss_regTrMagWt0_regRotAngMagWt0_386to386rather4096_depthScratchDAM_motionDftFlow_allRegLoss_doubleB_loss2weight100_tanhandclampSample_deltaMax001/0918-212235/models/weights_4'
traiend_models_root = '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/new_batch/b3accu4_scaredExcelSM_MonFast3r_trainedNfzD/1010-145110/models/weights_4'
traiend_models_root = '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/new_batch/b3accu4_scaredcorrectedExcelSM_posetr_fzenc_trainedNfzD/1013-104014/models/weights_3'
trained_models_root = '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/new_batch/b3accu4_scaredcorrectedExcelSM_posetr_fzenc_trainedNfzD_dft_DAonly/1014-185504/models/weights_4'
trained_models_root = '/mnt/nct-zfs/TCO-Test/jinjingxu/exps/train/mvp3r/results/unisfm/new_batch/b3accu4_scaredcorrectedExcelSM_posetr_fzenc_trainedNfzD_learnedMskTrePx1_learnedlossR2Weight1_reproj2only/1014-173620/models/weights_4'


print('pose_model_type:', opts.pose_model_type)
if opts.pose_model_type == "separate_resnet":
    pose_encoder_path = os.path.join(traiend_models_root, "pose_encoder.pth")
    pose_decoder_path = os.path.join(traiend_models_root, "pose.pth")
    assert os.path.exists(pose_encoder_path), f"pose_encoder_path {pose_encoder_path} does not exist"
    assert os.path.exists(pose_decoder_path), f"pose_decoder_path {pose_decoder_path} does not exist"
    trainer.models["pose_encoder"].load_state_dict(torch.load(pose_encoder_path))
    print(f"loaded pose_encoder from {pose_encoder_path}")
    trainer.models["pose"].load_state_dict(torch.load(pose_decoder_path))
    print(f"loaded pose_decoder from {pose_decoder_path}")
    # opts.enable_motion_computation = True


    # load pretrained depth model
    depth_model_path = os.path.join(traiend_models_root, "depth_model.pth")
    depth_model_dict = torch.load(depth_model_path)
    trainer.models["depth_model"].load_state_dict(depth_model_dict)
    print(f"loaded depth_model from {depth_model_path}")

    # # load pretrained motion flow model
    # motion_flow_model_path = os.path.join(traiend_models_root, "motion_raft_flow.pth")
    # motion_flow_model_dict = torch.load(motion_flow_model_path)
    # print(trainer.models.keys())
    # # trainer.models["motion_raft_flow"].load_state_dict(motion_flow_model_dict)
    # # trainer.models["raft_flow"].load_state_dict(motion_flow_model_dict)
    # print(f"loaded motion_flow_model from {motion_flow_model_path}")
elif opts.pose_model_type == "endofast3r" or opts.pose_model_type == "posetr_net":
    # load trained models for evaluation
    # refer to how resumed training is implemented in train_end_to_end.py
    
    # Load all models from the trained_models_root directory
    for model_name, model in trainer.models.items():
        model_path = os.path.join(traiend_models_root, f"{model_name}.pth")
        if os.path.exists(model_path):
            print(f"Loading {model_name} weights...")
            model_dict = model.state_dict()
            pretrained_dict = torch.load(model_path)
            pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
            model_dict.update(pretrained_dict)
            model.load_state_dict(model_dict)
        else:
            print(f"Warning: {model_name}.pth not found, skipping...")
    
    print("EndoFASt3r model loading completed!")

else:
    assert False, "pose_model_type should be separate_resnet or posetr_net"




In [ ]:
# opts.pose_model_type
trainer.opt.pose_model_type,trainer.opt.unireloc3r_pose_estimation_mode,trainer.opt.enable_all_depth,trainer.opt.enable_motion_computation,trainer.opt.enable_mutual_motion

# trainer = Trainer(opts)
self = trainer #Trainer(opts)


# trainer.train()
"""Run the entire training pipeline
"""
self.epoch = 0
self.step = 0
self.start_time = time.time()


In [ ]:
# force no shuffle when test
from torch.utils.data import DataLoader
self.train_dataset.is_train = False # avoid flip


# avoid shuffle
self.train_loader = DataLoader(
    self.train_dataset, self.opt.batch_size, 
    shuffle=False,
    num_workers=self.opt.num_workers, pin_memory=True, drop_last=True)

In [ ]:
# # trainer = Trainer(opts)
# self = trainer #Trainer(opts)


# # trainer.train()
# """Run the entire training pipeline
# """
# self.epoch = 0
# # self.step = 0
# self.start_time = time.time()

#quick and convenient but confusing recontrooling!
# self.opt.min_depth = 10.0 #risky
# self.opt.min_depth = 0.1 #risky
# self.opt.max_depth = 150.0
# self.opt.gt_metric_rel_pose_as_estimates_debug = False

# trained fast3r
# depth: 0.2-0.3
# rel_pose_per_axis: 2e-4

with torch.no_grad():
    # for self.epoch in range(self.opt.num_epochs):
    # self.run_epoch(self.epoch)
    """Run a single epoch of training and validation
    """
    epoch = self.epoch
    print("Training")

    self.model_optimizer_0.zero_grad()# debug only: save_mem
    self.model_optimizer.zero_grad()# debug only: save_mem

    for batch_idx, inputs in enumerate(self.train_loader):
    # while True:

        # train_iter = iter(self.train_loader)
        # inputs = next(train_iter)



        before_op_time = time.time()

        # position
        self.set_train_0()
        if self.opt.debug:
            self.freeze_params(keys = ['position_encoder'])#debug only to save_mem
        _, losses_0 = self.process_batch_0(inputs)
        self.model_optimizer_0.zero_grad()
        # losses_0["loss"].backward()#debug only
        torch.nn.utils.clip_grad_norm_(self.parameters_to_train_0, max_norm=1.0)
        self.model_optimizer_0.step()

        self.set_train()
        if self.opt.debug:
            self.freeze_params(keys = ['depth_model', 'pose', 'transform_encoder'])#debug only to save_mem
        outputs, losses= self.process_batch(inputs)
        self.model_optimizer.zero_grad()
        # losses["loss"].backward() #debug only
        torch.nn.utils.clip_grad_norm_(self.parameters_to_train, max_norm=1.0)
        self.model_optimizer.step()

        duration = time.time() - before_op_time

        # phase = batch_idx % self.opt.log_frequency == 0

        # # if phase:
        # # self.log_time(batch_idx, duration, losses["loss"].cpu().data)
        # self.log_time(0, duration, losses["loss"].cpu().data, losses_0["loss"].cpu().data, )
        # scalers_to_log = {
        #     "scalar/loss": losses["loss"],
        #     "scalar/loss_0": losses_0["loss"],
        #     # "scalar/trans_err": metric_errs["trans_err"].mean(),
        #     # "scalar/rot_err": metric_errs["rot_err"].mean()
        # }
        # for k, v in errs.items():
        #     print(f"scalar/{k}: {v}")
        #     scalers_to_log[f"scalar/{k}"] = v
        # concat_img, img_order_strs = self.log("train", inputs, outputs, scalers_to_log, compute_vis=True, online_vis=False)
        
        
        # Use the accumulated loss for logging (multiply back to show effective loss)
        # effective_loss = losses["loss"] * (self.opt.accumulate_steps / max(1, accumulate_step % self.opt.accumulate_steps))
        # self.log_time(batch_idx, duration, effective_loss.cpu().data)
        losses_to_log = {
            "loss": losses["loss"],
            "loss_0": losses_0["loss"],
        }
        errs_to_log = {}
        errs = self.compute_pose_metrics(inputs, outputs)
        for k, v in errs.items():
            assert len(v) == len(self.opt.frame_ids)-1, f'{k}: {v}'
            # v is a list of torch tensors, average over them
            errs_to_log[f"{k}"] = torch.mean(torch.stack(v)).item()

        self.log_time(self.step, duration, losses["loss"].cpu().data, losses_0["loss"].cpu().data, 
                        errs_to_log)
        scalers_to_log = {**losses_to_log, **errs_to_log}
        scalers_to_log = {f"scalar/{k}": v for k, v in scalers_to_log.items()}    
        # #compute_pose_err
        for k, v in errs.items():
            print(f"{k}: {v}")

        # print('trans_err:', outputs[("trans_err", 0)])
        # print('rot_err:', outputs[("rot_err", 0)])



        # concat_img, img_order_strs = self.log("train", inputs, outputs, 
        concat_img_list, img_order_strs_list = self.log("train", inputs, outputs, 
                    scalers_to_log, 
                    compute_vis=True)

        import os, cv2
        save_path = "/mnt/cluster/workspaces/jinjingxu/proj/UniSfMLearner/submodule/Endo_FASt3r/tmp"
        os.makedirs(save_path, exist_ok=True)
        for concat_img, img_order_strs in zip(concat_img_list, img_order_strs_list):
        # concat_img, img_order_strs = self.log("train", inputs, outputs, 

            # remove the first char before the first _
            # img_order_strs = img_order_strs.split('_')[1:]
            # img_order_strs = '_'.join(img_order_strs)
            # full_save_path = os.path.join(save_path, f"{img_order_strs}.png")
            full_save_path = os.path.join(save_path, f"{img_order_strs}_{self.step}.png")
            cv2.imwrite(full_save_path, concat_img)
            print(f"saved train_concat_img.png in {full_save_path}")

        self.step += 1
        
        # break
            
        self.model_lr_scheduler.step()
        self.model_lr_scheduler_0.step()

        # if (self.epoch + 1) % self.opt.save_frequency == 0:
        #     self.save_model()
        # break
    


    # depth dim:
    # 256 320
    # 128,160,  
    # 64,80,
    # 32 40    # ===we go for this



# 0.06 0.04 0.04

In [ ]:
# 97 -16 -43

In [ ]:

# self.set_eval()
# try:
#     # inputs = self.val_iter.next()
#     inputs = next(self.val_iter)
# except StopIteration:
#     self.val_iter = iter(self.val_loader)
#     inputs = self.val_iter.next()

# with torch.no_grad():
#     # outputs, losses = self.process_batch_val(inputs)
#     """Pass a minibatch through the network and generate images and losses
#     """
#     for key, ipt in inputs.items():
#         inputs[key] = ipt.to(self.device)

#     if self.opt.pose_model_type == "shared":
#         # If we are using a shared encoder for both depth and pose (as advocated
#         # in monodepthv1), then all images are fed separately through the depth encoder.
#         print('CHECK: self.opt.pose_model_type == "shared"')
#         all_color_aug = torch.cat([inputs[("color_aug", i, 0)] for i in self.opt.frame_ids])
#         all_features = self.models["encoder"](all_color_aug)
#         all_features = [torch.split(f, self.opt.batch_size) for f in all_features]

#         features = {}
#         for i, k in enumerate(self.opt.frame_ids):
#             features[k] = [f[i] for f in all_features]

#         outputs = self.models["depth"](features[0])
#     else:
#         print('CHECK: self.opt.pose_model_type != "shared"')
#         # Otherwise, we only feed the image with frame_id 0 through the depth encoder
#         features = self.models["encoder"](inputs["color_aug", 0, 0])
#         outputs = self.models["depth"](features)

#     if self.opt.predictive_mask:
#         outputs["predictive_mask"] = self.models["predictive_mask"](features)

#     if self.use_pose_net:
#         outputs.update(self.predict_poses(inputs, features, outputs))

#     self.generate_images_pred(inputs, outputs)
#     losses = self.compute_losses_val(inputs, outputs)

#     # return outputs, losses

#     self.log("val", inputs, outputs, losses)
#     del inputs, outputs, losses